In [8]:
import os
import sys
import numpy as np
import yaml
import argparse
import shutil
from copy import deepcopy
from os.path import join as pjoin

BASEPATH = os.path.dirname('/home/taehyun/workspace/personal_git/deep-motion-editing')
sys.path.insert(0, '/home/taehyun/workspace/personal_git/deep-motion-editing')

# sys.path.insert(0, pjoin(BASEPATH))
# # sys.path.insert(0, pjoin(BASEPATH, '..', '..'))
from utils.animation_data import AnimationData
from utils.load_skeleton import Skel

In [11]:
dataset_config = '/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/global_info/xia_dataset.yml'
with open(dataset_config, "r") as f:
    cfg = yaml.load(f, Loader=yaml.Loader)
content_namedict = [full_name.split('_')[0] for full_name in cfg["content_full_names"]]
content_test_cnt = cfg["content_test_cnt"]
content_names = cfg["content_names"]
style_names = cfg["style_names"]
style_name_to_idx = {name: i for i, name in enumerate(style_names)}


In [19]:
def get_bvh_files(directory):
    return [os.path.join(directory, f) for f in sorted(list(os.listdir(directory)))
            if os.path.isfile(os.path.join(directory, f))
            and f.endswith('.bvh') and f != 'rest.bvh']



In [22]:
skel = Skel()
bvh_path = '/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia'
bvh_files = get_bvh_files(bvh_path)

train_inputs = []
test_inputs = []
trainfull_inputs = []
test_files = []
test_cnt = {}  # indexed by content_style

In [75]:
def motion_and_phase_to_dict(fulls, style, meta):
    """
    fulls: a list of [T, xxx + 1] - motion and phase
    style: a *number*
    meta: a dict, e.g. {"style": "angry", "content": "walk"}
    """
    output = []
    for full in fulls:
        motion, phase = full[:, :-1], full[:, -1]
        phase_label = phase[len(phase) // 2]
        meta_copy = deepcopy(meta)
        meta_copy["phase"] = phase_label
        output.append({
            "motion": motion,
            "style": style,
            "meta": meta_copy
        })
    return output
def process_file(filename, divider, window, window_step, downsample=4, skel=None, divide=True):
    input = bvh_to_motion_and_phase(filename, downsample=downsample, skel=skel)  # [T, xxx]
    return divider(input, window=window, window_step=window_step, divide=divide)

def divide_clip_xia(input, window, window_step, divide):
    if not divide:  # return the whole clip
        t = ((input.shape[0]) // 4) * 4 + 4
        t = max(t, 12)
        if len(input) < t:
            input = pad_to_window(input, t)
        return [input]

    windows = []
    j = -(window // 4)
    total = len(input)
    while True:
        slice = input[max(j, 0): j + window].copy()  # remember to COPY!!
        if len(slice) < window:
            slice = pad_to_window(slice, window)
        windows.append(slice)
        j += window_step
        if total - j < (3 * window) // 4:
            break
    return windows
def pad_to_window(slice, window):
    def get_reflection(src, tlen):  # return [src-reversed][src][src-r]...
        x = src.copy()
        x = np.flip(x, axis=0)
        ret = x.copy()
        while len(ret) < tlen:
            x = np.flip(x, axis=0)
            ret = np.concatenate((ret, x), axis=0)
        ret = ret[:tlen]
        return ret

    if len(slice) >= window:
        return slice
    left_len = (window - len(slice)) // 2 + (window - len(slice)) % 2
    right_len = (window - len(slice)) // 2
    left = np.flip(get_reflection(np.flip(slice, axis=0), left_len), axis=0)
    right = get_reflection(slice, right_len)
    slice = np.concatenate([left, slice, right], axis=0)
    assert len(slice) == window
    return slice

def bvh_to_motion_and_phase(filename, downsample, skel):
    anim = AnimationData.from_BVH(filename, downsample=downsample, skel=skel)
    full = anim.get_full()  # [T, xxx]
    print(full.shape)
    phases = anim.get_phases()  # [T, 1]
    return np.concatenate((full, phases), axis=-1)

def set_init(dic, key, value):
    try:
        dic[key]
    except KeyError:
        dic[key] = value



In [76]:
window = 32
window_step=8
for i, item in enumerate(bvh_files):
    print('Processing %i of %i (%s)' % (i, len(bvh_files), item))
    filename = item.split('/')[-1]
    style, content_idx, _ = filename.split('_')
    content = content_namedict[int(content_idx) - 1]
    content_style = "%s_%s" % (content, style)
    
    uclip = motion_and_phase_to_dict(process_file(item, divider=divide_clip_xia, window=window, window_step=window_step,
                                                  skel=skel, divide=False),
                                     style_name_to_idx[style],
                                     {"style": style, "content": content})
    # Whether this should be a test clip
    set_init(test_cnt, content_style, 0)
    if test_cnt[content_style] < content_test_cnt[content]:
        test_cnt[content_style] += 1
        test_inputs += uclip
        test_files.append(filename)
    else:
        trainfull_inputs += uclip
        clips = motion_and_phase_to_dict(process_file(item, divider=divide_clip_xia, window=window, window_step=window_step,
                                                      skel=skel, divide=True),
                                         style_name_to_idx[style],
                                         {"style": style, "content": content})
        train_inputs += clips


Processing 0 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_01_000.bvh)
(90, 132)
(90, 132)
Processing 1 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_01_001.bvh)
(68, 132)
(68, 132)
Processing 2 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_01_002.bvh)
(70, 132)
(70, 132)
Processing 3 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_01_003.bvh)
(69, 132)
(69, 132)
Processing 4 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_02_000.bvh)
(57, 132)
(57, 132)
Processing 5 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_02_001.bvh)
(55, 132)
(55, 132)
Processing 6 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_02_002.bvh

(21, 132)
Processing 58 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_20_001.bvh)
(17, 132)
(17, 132)
Processing 59 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_20_002.bvh)
(18, 132)
(18, 132)
Processing 60 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_21_000.bvh)
(20, 132)
(20, 132)
Processing 61 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_21_001.bvh)
(18, 132)
(18, 132)
Processing 62 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_21_002.bvh)
(20, 132)
(20, 132)
Processing 63 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/angry_22_000.bvh)
(38, 132)
(38, 132)
Processing 64 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia

(25, 132)
Processing 122 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/childlike_19_000.bvh)
(24, 132)
(24, 132)
Processing 123 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/childlike_19_001.bvh)
(26, 132)
(26, 132)
Processing 124 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/childlike_19_002.bvh)
(26, 132)
(26, 132)
Processing 125 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/childlike_20_000.bvh)
(20, 132)
(20, 132)
Processing 126 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/childlike_20_001.bvh)
(18, 132)
(18, 132)
Processing 127 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/childlike_20_002.bvh)
(19, 132)
(19, 132)
Processing 128 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editin

(33, 132)
Processing 178 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/depressed_12_002.bvh)
(36, 132)
(36, 132)
Processing 179 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/depressed_13_000.bvh)
(48, 132)
(48, 132)
Processing 180 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/depressed_13_001.bvh)
(47, 132)
(47, 132)
Processing 181 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/depressed_13_002.bvh)
(32, 132)
(32, 132)
Processing 182 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/depressed_13_003.bvh)
(33, 132)
(33, 132)
Processing 183 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/depressed_14_000.bvh)
(17, 132)
(17, 132)
Processing 184 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editin

(43, 132)
Processing 237 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_08_000.bvh)
(43, 132)
(43, 132)
Processing 238 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_08_001.bvh)
(29, 132)
(29, 132)
Processing 239 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_08_002.bvh)
(45, 132)
(45, 132)
Processing 240 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_09_000.bvh)
(24, 132)
(24, 132)
Processing 241 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_10_000.bvh)
(23, 132)
(23, 132)
Processing 242 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_10_001.bvh)
(24, 132)
(24, 132)
Processing 243 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_tran

(21, 132)
Processing 292 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_26_000.bvh)
(16, 132)
(16, 132)
Processing 293 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_26_001.bvh)
(6, 132)
(6, 132)
Processing 294 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_27_000.bvh)
(12, 132)
(12, 132)
Processing 295 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/neutral_28_000.bvh)
(19, 132)
(19, 132)
Processing 296 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/old_01_000.bvh)
(95, 132)
(95, 132)
Processing 297 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/old_01_001.bvh)
(85, 132)
(85, 132)
Processing 298 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/

(25, 132)
(25, 132)
Processing 350 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/old_19_000.bvh)
(27, 132)
(27, 132)
Processing 351 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/old_19_001.bvh)
(32, 132)
(32, 132)
Processing 352 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/old_19_002.bvh)
(28, 132)
(28, 132)
Processing 353 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/old_22_000.bvh)
(21, 132)
(21, 132)
Processing 354 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/old_22_001.bvh)
(22, 132)
(22, 132)
Processing 355 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/old_22_002.bvh)
(23, 132)
(23, 132)
Processing 356 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/moca

Processing 420 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/proud_22_001.bvh)
(31, 132)
(31, 132)
Processing 421 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/proud_22_002.bvh)
(27, 132)
(27, 132)
Processing 422 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/proud_23_000.bvh)
(30, 132)
(30, 132)
Processing 423 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/proud_23_001.bvh)
(27, 132)
(27, 132)
Processing 424 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/proud_24_000.bvh)
(23, 132)
(23, 132)
Processing 425 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/proud_26_000.bvh)
(5, 132)
(5, 132)
Processing 426 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/prou

(24, 132)
(24, 132)
Processing 480 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/sexy_22_000.bvh)
(32, 132)
(32, 132)
Processing 481 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/sexy_22_001.bvh)
(35, 132)
(35, 132)
Processing 482 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/sexy_22_002.bvh)
(37, 132)
(37, 132)
Processing 483 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/sexy_22_003.bvh)
(35, 132)
(35, 132)
Processing 484 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/sexy_23_000.bvh)
(35, 132)
(35, 132)
Processing 485 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/sexy_23_001.bvh)
(34, 132)
(34, 132)
Processing 486 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/dat

(58, 132)
Processing 535 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/strutting_13_001.bvh)
(44, 132)
(44, 132)
Processing 536 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/strutting_13_002.bvh)
(48, 132)
(48, 132)
Processing 537 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/strutting_13_003.bvh)
(46, 132)
(46, 132)
Processing 538 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/strutting_14_000.bvh)
(18, 132)
(18, 132)
Processing 539 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/strutting_14_001.bvh)
(17, 132)
(17, 132)
Processing 540 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editing/style_transfer/data/mocap_xia/strutting_14_002.bvh)
(17, 132)
(17, 132)
Processing 541 of 572 (/home/taehyun/workspace/personal_git/deep-motion-editin

In [110]:
trainfull_inputs[102]['motion'].shape

(44, 132)

In [46]:
data_dict = {}
data_info = {}
for subset, inputs in zip(["train", "test", "trainfull"], [train_inputs, test_inputs, trainfull_inputs]):
    motions = [input["motion"] for input in inputs]
    styles = [input["style"] for input in inputs]
    meta = {key: [input["meta"][key] for input in inputs] for key in inputs[0]["meta"].keys()}
    data_dict[subset] = {"motion": motions, "style": styles, "meta": meta}

    """compute meta info"""
    num_clips = len(motions)
    info = {"num_clips": num_clips,
            "distribution":
                {style:
                     {content: len([i for i in range(num_clips) if meta["style"][i] == style and meta["content"][i] == content])
                      for content in content_names}
                 for style in style_names}
            }
    data_info[subset] = info


In [51]:
motions[0].shape

(72, 132)

In [52]:
for subset, inputs in zip(["train", "test", "trainfull"], [train_inputs, test_inputs, trainfull_inputs]):
    print(inputs)


[{'motion': array([[ 0.97820418,  0.20761797,  0.00108462, ...,  0.        ,
         1.        ,  1.        ],
       [ 0.97821722,  0.2074824 , -0.00308633, ...,  0.        ,
         1.        ,  1.        ],
       [ 0.97797899,  0.20829438, -0.01189335, ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [-0.9828125 , -0.1664647 ,  0.07943917, ...,  0.        ,
         0.        ,  1.        ],
       [-0.98450751, -0.15782188,  0.07612639, ...,  0.        ,
         1.        ,  1.        ],
       [-0.98553001, -0.15331381,  0.07226846, ...,  0.        ,
         1.        ,  1.        ]]), 'style': 0, 'meta': {'style': 'angry', 'content': 'walk', 'phase': 4.71238898038469}}, {'motion': array([[ 0.97786881,  0.2081272 , -0.02018552, ...,  1.        ,
         1.        ,  1.        ],
       [ 0.97797899,  0.20829438, -0.01189335, ...,  1.        ,
         1.        ,  1.        ],
       [ 0.97821722,  0.2074824 , -0.00308633, ...,  0.        ,
         1.

In [61]:
data_dict['train']['motion'][0].shape

(32, 132)

In [70]:
data_dict['trainfull']

{'motion': [array([[ 0.97797899,  0.20829438, -0.01189335, ...,  1.        ,
           1.        ,  1.        ],
         [ 0.97786881,  0.2081272 , -0.02018552, ...,  1.        ,
           1.        ,  1.        ],
         [ 0.97786881,  0.2081272 , -0.02018552, ...,  1.        ,
           1.        ,  1.        ],
         ...,
         [-0.99039518, -0.13071758,  0.04406493, ...,  0.        ,
           0.        ,  1.        ],
         [-0.99031789, -0.13422665,  0.03534718, ...,  0.        ,
           0.        ,  1.        ],
         [-0.99031789, -0.13422665,  0.03534718, ...,  0.        ,
           0.        ,  1.        ]]),
  array([[ 0.96933996,  0.24488619, -0.01669637, ...,  1.        ,
           0.        ,  1.        ],
         [ 0.96950563,  0.24401696, -0.01982334, ...,  1.        ,
           0.        ,  1.        ],
         [ 0.96950563,  0.24401696, -0.01982334, ...,  1.        ,
           0.        ,  1.        ],
         ...,
         [ 0.98830416,  